<a href="https://colab.research.google.com/github/nikhilrathaur/Generating-Harry-Potter-Story-Using-RNN-and-Tensorflow/blob/master/Generating_Harry_Potter_Story_Using_RNN_and_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 'https://drive.google.com/uc?id=1gpfiJNP0A7r6z_LGUENv6dFWGnMPl0Ga'

Downloading...
From: https://drive.google.com/uc?id=1gpfiJNP0A7r6z_LGUENv6dFWGnMPl0Ga
To: /content/144820_338363_bundle_archive.zip
2.39MB [00:00, 76.6MB/s]


In [2]:
!unzip '/content/144820_338363_bundle_archive.zip'

Archive:  /content/144820_338363_bundle_archive.zip
  inflating: 1SorcerersStone.txt     
  inflating: 2ChamberofSecrets.txt   
  inflating: 3ThePrisonerOfAzkaban.txt  
  inflating: 4TheGobletOfFire.txt    
  inflating: 5OrderofthePhoenix.txt  
  inflating: 6TheHalfBloodPrince.txt  
  inflating: 7DeathlyHollows.txt     


In [23]:
import tensorflow as tf
import numpy as np
import os
import time

In [34]:
files= ['1SorcerersStone.txt', '2ChamberofSecrets.txt', '3ThePrisonerOfAzkaban.txt', '4TheGobletOfFire.txt', '5OrderofthePhoenix.txt', '6TheHalfBloodPrince.txt', '7DeathlyHollows.txt']
with open('harrypotter.txt', 'w') as outfile:
  for file in files:
    with open(file) as infile:
      outfile.write(infile.read())
  text = open('harrypotter.txt').read()

In [35]:
print(text[:300])

Harry Potter and the Sorcerer's Stone 

CHAPTER ONE 

THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they 


In [36]:
vocab = sorted(set(text))
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])
#how it looks:
print ('{} -- characters mapped to int -- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Harry Potter ' -- characters mapped to int -- > [39 64 81 81 88  3 47 78 83 83 68 81  3]


In [37]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(data):
  input_text = data[:-1]
  target_text = data[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)

In [38]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [39]:
vocab_size = len(vocab)
embedding_dim = 300
# Number of RNN units 
rnn_units1 = 512
rnn_units2 = 256
rnn_units= [rnn_units1, rnn_units2]
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
       batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
model = build_model(
vocab_size = vocab_size,
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

In [40]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
         logits, from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [41]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix, save_weights_only=True)

In [42]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
latest_check = tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/50
967/967 [==============================] - 38s 39ms/step - loss: 1.7533 - accuracy: 0.4992
Epoch 2/50
967/967 [==============================] - 39s 40ms/step - loss: 1.2926 - accuracy: 0.6140
Epoch 3/50
967/967 [==============================] - 39s 40ms/step - loss: 1.2228 - accuracy: 0.6319
Epoch 4/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1880 - accuracy: 0.6412
Epoch 5/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1653 - accuracy: 0.6473
Epoch 6/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1493 - accuracy: 0.6519
Epoch 7/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1365 - accuracy: 0.6553
Epoch 8/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1262 - accuracy: 0.6581
Epoch 9/50
967/967 [==============================] - 39s 40ms/step - loss: 1.1180 - accuracy: 0.6604
Epoch 10/50
967/967 [==============================] - 39s 40ms/step - loss: 1.110

In [43]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(latest_check)
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 300)            31800     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 256)            591360    
_________________________________________________________________
dense_3 (Dense)              (1, None, 106)            27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [50]:
def generate_text(model, start_string):
  num_generate = 1000  #can be anything you like
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  scaling = 0.5 #kept at a lower value here
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / scaling
    predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(index2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [51]:
print(generate_text(model, start_string="Severus Snape"))

Severus Snape was methe they had been thinking about it. The Weasleys wand still in a state. Harry found himself being stuck in the stairs and saw that she could not be able to stop the minutes before the rather stupid black hair creaked away from the little cold and interested in a very little glove. He stood up, and he saw the sound of the place where he looked at each other, the boy who had already stood there with the cart and started sickening as he spoke, saw him so hard that he had been crammed into the castle, then she put her hair again, and she was came to find himself. "It has happened to you? I am going to be friends... You haven’t been to know where this could just continue my father was a little one of the wand - he had not been happy at all, the distant bunch of green light and he stood up, and Harry saw the locket into the trees. He was so close that they were all still absolute at the same time. He seemed to have been there, Harry thought he was working to the best to 